Generate text embeddings

Text embeddings convert text into numerical vectors that capture semantic meaning. Use them for semantic search, similarity calculations, and other NLP tasks.

In [ ]:
pip install -U "daft[sentence-transformers]"

In [ ]:
import daft
from daft.functions.ai import embed_text

(
    daft.read_huggingface("togethercomputer/RedPajama-Data-1T")
    .with_column("embedding", embed_text(daft.col("text")))
    .show()
)

Using Sentence Transformers

In [ ]:
import daft
from daft.functions.ai import embed_text

provider = "sentence_transformers"
model = "BAAI/bge-base-en-v1.5"

(
    daft.read_huggingface("togethercomputer/RedPajama-Data-1T")
    .with_column("embedding", embed_text(daft.col("text"), provider=provider, model=model))
    .show()
)

How to work with embeddings

Chunk text into smaller pieces

When working with large text documents, you often need to break them into smaller chunks

Use spaCy for sentence Chunking

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import daft
import typing

nlp_model_name = "en_core_web_sm"

@daft.func
def chunk_by_sentences(text: str) -> typing.Iterator[str]:
    import spacy
    nlp = spacy.load(nlp_model_name)
    doc = nlp(text)
    for sentence in doc.sents:
        yield sentence.text

(
    daft.read_huggingface("roneneldan/TinyStories")
    .limit(8)
    .with_column("chunks", chunk_by_sentences(daft.col("text")))
    .show()
)


textString,chunksString
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.Lily went to her mom and said, ""Mom, I found this needle. Can you share it with me and sew my shirt?"" Her mom smiled and said, ""Yes, Lily, we can share the needle and fix your shirt.""Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.","One day, a little girl named Lily found a needle in her room."
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.Lily went to her mom and said, ""Mom, I found this needle. Can you share it with me and sew my shirt?"" Her mom smiled and said, ""Yes, Lily, we can share the needle and fix your shirt.""Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.",She knew it was difficult to play with it because it was sharp.
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.Lily went to her mom and said, ""Mom, I found this needle. Can you share it with me and sew my shirt?"" Her mom smiled and said, ""Yes, Lily, we can share the needle and fix your shirt.""Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.","Lily wanted to share the needle with her mom, so she could sew a button on her shirt."
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.Lily went to her mom and said, ""Mom, I found this needle. Can you share it with me and sew my shirt?"" Her mom smiled and said, ""Yes, Lily, we can share the needle and fix your shirt.""Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.","Lily went to her mom and said, ""Mom, I found this needle."
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.Lily went to her mom and said, ""Mom, I found this needle. Can you share it with me and sew my shirt?"" Her mom smiled and said, ""Yes, Lily, we can share the needle and fix your shirt.""Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.","Can you share it with me and sew my shirt?"""
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily 

🗡️ 🐟 Limit 8: 00:00 

Working with Images

Daft is built to work comfortably with images. This guide shows you how to accomplish common image processing tasks with Daft:

Downloading and decoding images
Generate image embeddings
Classify images
It also explains some concepts on Dynamic execution for multimodal workloads to improve your mental model of how the Daft engine works.

In [ ]:

import daft
from daft.functions.ai import embed_image

(
    daft.read_huggingface("xai-org/RealworldQA")
    .with_column("image", daft.col("image")["bytes"].image.decode())
    .with_column("embedding", embed_image(daft.col("image")))
    .show()
)

imageImage[MIXED],questionString,answerString,embeddingEmbedding[Float32; 512]
""" />",Which of the 3 objects is the smallest?A. The object on the right is the smallest object.B. The object on the left is the smallest object.C. The object in the middle is the smallest object.Please answer directly with only the letter of the correct option and nothing else.,C,▂▂▂▂█▂▂▂▂▂▂▃▂▃▃▂▂▂▂▂▁▂▁▁
""" />",Is the closest red car to us closer than the closest semi truck?Please answer directly with a single word or number.,Yes,▂▁▂▂█▁▂▁▂▁▂▂▁▂▂▁▂▂▁▂▁▁▁▁
""" />",Can the car directly ahead of us drive straight through the intersection?A. YesB. NoPlease answer directly with only the letter of the correct option and nothing else.,B,▂▂▂▂█▂▂▁▂▂▁▂▁▁▁▁▂▂▂▁▂▂▁▂
""" />",What color is the traffic lights in this scene?Please answer directly with a single word or number.,Green,▂▂▂▂█▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂
""" />",Which direction are the knobs currently in?A. The knobs are currently in the downward facing position.B. The knobs are currently in the right facing position.C. The knobs are currently in the upward facing position.Please answer directly with only the letter of the correct option and nothing else.,C,▂▂▂▂█▁▂▂▂▂▂▃▁▂▃▁▂▂▂▂▂▂▁▁
""" />","Which is closer to us, the bus or the sedan?Please answer directly with a single word or number.",Bus,▂▁▂▂█▂▂▁▁▁▁▂▁▁▂▁▂▁▁▂▂▂▁▂
""" />","Based on the shadow, where is the sun relative to us?A. BehindB. On the rightC. In frontPlease answer directly with only the letter of the correct option and nothing else.",A,▂▂▂▂█▂▂▁▂▁▁▂▂▂▂▁▂▂▂▁▂▂▁▂
""" />",How many cyclists are visible?Please answer directly with a single word or number.,2,▂▂▂▂█▁▂▁▂▂▁▂▁▁▂▁▂▂▂▁▁▂▂▂


In [ ]:
!pip install -U daft[ai] transformers pillow


In [ ]:
import daft
from transformers import pipeline
from PIL import Image
import io

df = daft.from_pydict({
    "image_url": [
        "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png",
        "https://upload.wikimedia.org/wikipedia/commons/a/a3/81_INF_DIV_SSI.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg",
    ]
})

df = df.with_column("image_bytes", df["image_url"].url.download())


df = df.with_column("image", df["image_bytes"].image.decode())


image_classifier = pipeline("image-classification", model="google/vit-base-patch16-224")


@daft.func
def classify_image(file_bytes: bytes) -> str:
    image = Image.open(io.BytesIO(file_bytes)).convert("RGB")
    result = image_classifier(image)[0]
    return f"{result['label']} ({result['score']:.2f})"


df = df.with_column("classification", classify_image(df["image_bytes"]))

df.collect()
df.select("image", "classification").show()


imageImage[MIXED],classificationString
""" />","pug, pug-dog (0.85)"
""" />","saltshaker, salt shaker (0.09)"
""" />",schipperke (0.43)
""" />",tiger cat (0.44)


JSON

In [ ]:

df = daft.from_pydict({
    "json": [
        '{"a": 1, "b": 2}',
        '{"a": 3, "b": 4}',
    ],
})
df = df.with_column("a", df["json"].jq(".a"))
df.collect()

🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

jsonString,aString
"{""a"": 1, ""b"": 2}",1
"{""a"": 3, ""b"": 4}",3


Extracting and Flattening Nested Data

When working with nested data---like log files, metadata, deserialized JSON---we often need to extract specific fields or flatten the entire structure into individual columns. Daft provides two main approaches for this:

Extracting specific fields: Using the [] operator to access nested fields
Flattening all fields: Using .unnest() or the * wildcard to expand all nested fields into separate columns

In [ ]:
import daft
from daft import col

swe_rebench_metadata = daft.read_parquet("hf://datasets/nebius/SWE-rebench/data/*.parquet").select("meta")
swe_rebench_metadata.schema()

column_name,type
meta,"Struct[commit_name: String, failed_lite_validators: List[String], has_test_patch: Bool, is_lite: Bool, llm_score: Struct[difficulty_score: Int64, issue_text_score: Int64, test_score: Int64], num_modified_files: Int64]"


Extract a specific field from the struct by using the [] operator

In [ ]:
swe_rebench_metadata.select(col("meta")["llm_score"]["difficulty_score"]).show()

🗡️ 🐟 Project: 00:00 

difficulty_scoreInt64
2
1
2
2
0
0
1
0


To extract all the nested columns, we can use the .unnest() expression or the wildcard * to access all fields of the meta struct column.

In [ ]:
swe_rebench_metadata.select(daft.col("meta").unnest()).show()


commit_nameString,failed_lite_validatorsList[String],has_test_patchBool,is_liteBool,"llm_scoreStruct[difficulty_score: Int64, issue_text_score: Int64, test_score: Int64]",num_modified_filesInt64
head_commit,"[has_short_problem_statement, has_added_files, has_many_modified_files, has_many_hunks, has_pytest_match_arg]",true,false,"{difficulty_score: 2,issue_text_score: 1,test_score: 3,}",5
head_commit,"[has_many_modified_files, has_many_hunks]",true,false,"{difficulty_score: 1,issue_text_score: 0,test_score: 3,}",5
head_commit,"[has_removed_files, has_many_modified_files, has_many_hunks, has_pytest_match_arg]",true,false,"{difficulty_score: 2,issue_text_score: 3,test_score: 3,}",6
head_commit,[],true,true,"{difficulty_score: 2,issue_text_score: 0,test_score: 3,}",1
head_commit,[],true,true,"{difficulty_score: 0,issue_text_score: 3,test_score: 0,}",1
head_commit,[],true,true,"{difficulty_score: 0,issue_text_score: 0,test_score: 0,}",1
head_commit,[],true,true,"{difficulty_score: 1,issue_text_score: 0,test_score: 0,}",1
head_commit,"[has_hyperlinks, has_issue_reference, has_many_modified_files, has_many_hunks]",true,false,"{difficulty_score: 0,issue_text_score: 1,test_score: 2,}",3
